# 【補足】特徴量エンジニアリング

▼ 本章で使用するデータ  

- [ts_wide.csv](data/ts_wide.csv)


---

## 必要なモジュールのインポート

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = 15, 5  # 描画サイズの設定

# 警告非表示
warnings.filterwarnings('ignore')

## ワイドフォーマットとロングフォーマット

時系列データはデータの格納方法として**ワイドフォーマット**と**ロングフォーマット**があります。ワイドフォーマットはキーとなる変数を列として注目する変数の値を格納する形式です。  

**例**  

|  | 店舗 A | 店舗 B | 店舗 C |
| --- | --- | --- | --- |
| 2022-01-01 | 売上 | 売上 | 売上 |
| 2022-01-02 | 売上 | 売上 | 売上 |
| 2022-01-03 | 売上 | 売上 | 売上 |

またキーとなる変数を 1 つの列に格納し、注目する変数やその他の変数を 1 つのテーブルに格納できる形式をロングフォーマットと呼びます。  

**例**  

|  | 店舗 | 売上| 人数 |
| --- | --- | --- | --- |
| 2022-01-01 | A |  |  |
| 2022-01-01 | B |  |  |
| 2022-01-01 | C |  |  |
| 2022-01-02 | A |  |  |

本研修で使用する実データは**ロングフォーマット**で用意されています。本章の特徴量エンジニアリングを行う際はワイドフォーマットで扱う方が簡単です。  

まずはワイドフォーマットとロングフォーマットの変換方法を確認します。

In [2]:
# データの読み込み
df_wide = pd.read_csv(f'ts_wide.csv', index_col=0, parse_dates=True)
df_wide.head(3)

,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711


こちらのデータはワイドフォーマットです。ロングフォーマットへ変換します。

In [3]:
# ロングフォーマットへ変換
df_long = df_wide.stack().reset_index(1)
df_long.columns = ['id', 'value']
df_long.head(5)

,id,value
2016-07-01,A,532
2016-07-01,B,3314
2016-07-01,C,1136
2016-07-02,A,798
2016-07-02,B,2461


ロングフォーマットへ変換できました。それでは再度ワイドフォーマットへ変換してみましょう。

In [4]:
# ワイドフォーマットへ変換
df_wide = df_long.pivot(index=None, columns='id', values='value')
df_wide.head(3)

id,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711


ワイドフォーマットへ戻すことができました。上記のデータ操作はよく使用するので押さえておきましょう。

## カレンダー特徴量

それでは**特徴量エンジニアリング (Feature Engineering)** を行います。特徴量エンジニアリングとは今ある特徴量（データ）からドメイン知識などを活かして新しく特徴量を作成することです。これによりデータの質を上げ、機械学習モデルの予測性能を向上させることが目的になります。  

時系列データで行われる特徴量エンジニアリングは主に以下です。

- カレンダー特徴量
- ラグ特徴量
- ローリング特徴量
- エキスパディング特徴量
- リード特徴量

本章では上記それぞれの実装方法を紹介します。  

まずは**カレンダー特徴量**です。時系列データには、平日、週末、休日等、様々なカレンダー情報が紐付いています。このようなカレンダーなどに関する特徴量（説明変数）は、予測モデルの精度を高めることに大きく寄与することもあります。

In [5]:
# x に格納
x = df_wide
x.head(3)

id,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711


In [6]:
# 月の情報を確認
x.index.month

Int64Index([ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
            ...
            12, 12, 12, 12, 12, 12, 12, 12, 12, 12],
           dtype='int64', length=184)

上記を使用して、月の情報を特徴量として取得します。

In [7]:
# 何月かを特徴量として追加
x['month'] = x.index.month
x.head(3)

id,A,B,C,month
2016-07-01,532,3314,1136,7
2016-07-02,798,2461,1188,7
2016-07-03,823,3522,1711,7


月の情報を追加できました。またその日が祝日かどうかといった情報も特徴量として使用できます。こちらは `jpholiday` というライブラリを使用すると簡単に実装できます。

In [8]:
!pip install -q jpholiday
import jpholiday

In [9]:
# 祝日であるかを追加
x['is_holiday'] = x.index.map(jpholiday.is_holiday).astype(int)
x.head(3)

id,A,B,C,month,is_holiday
2016-07-01,532,3314,1136,7,0
2016-07-02,798,2461,1188,7,0
2016-07-03,823,3522,1711,7,0


祝日であるときは 1、それ以外は 0 というラベルをふることができました。このように様々なカレンダー特徴量を生成できます。特徴量を追加したことでデータの形式が変わったため、最初の状態に戻しておきます。

In [10]:
# データをもとに戻す
x = x.drop(['month', 'is_holiday'], axis=1)
x.head(3)

id,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711


## ラグ特徴量

ある t 期の売上高を予測する際に、前月の t-1 期の売上を特徴量として使用したり、前々月である t-2 期の売上高を特徴量として使用することがあります。t-1 期の売上高や t-2 期の売上高を**ラグ特徴量**と言います。

ラグ特徴量は目的変数に対して使用できますし、もちろん入力変数に対して使用しても構いません。

In [12]:
# 1 期前のlagを取得
x_lag1 = x.shift(1)
x_lag1.columns = ['A_lag1', 'B_lag1', 'C_lag1']
x_lag1.head(3)

,A_lag1,B_lag1,C_lag1
2016-07-01,NaN,NaN,NaN
2016-07-02,532.0,3314.0,1136.0
2016-07-03,798.0,2461.0,1188.0


In [13]:
# 7 期前のlag
x_lag7 = x.shift(7)
x_lag7.columns = ['A_lag7', 'B_lag7', 'C_lag7']
x_lag7.head(10)

,A_lag7,B_lag7,C_lag7
2016-07-01,NaN,NaN,NaN
2016-07-02,NaN,NaN,NaN
2016-07-03,NaN,NaN,NaN
2016-07-04,NaN,NaN,NaN
2016-07-05,NaN,NaN,NaN
2016-07-06,NaN,NaN,NaN
2016-07-07,NaN,NaN,NaN
2016-07-08,532.0,3314.0,1136.0
2016-07-09,798.0,2461.0,1188.0
2016-07-10,823.0,3522.0,1711.0


## ローリング特徴量

**ローリング特徴量**とは**過去の一定期間における集計値**を特徴量としたものです。月毎の時系列データであれば、t 期の売上高を予測する際に過去 3 ヶ月間の平均値や中央値、最大値や最小値を特徴量とすることです。

もちろん目的変数以外の入力変数にて、ローリング特徴量をとっても構いません。また集計する期間に関しても 1 つの期間（過去 3 ヶ月間など）だけでなく、過去3ヶ月・過去半年・過去 1 年間など、複数の期間を定めて作ることが多いです。

In [14]:
# 3期間の移動平均を算出
x_avg3 = x.rolling(window=3).mean() # 3 期間の終点でとる
x_avg3.columns = ['A_avg3', 'B_avg3', 'C_avg3']
x_avg3.head(3)

,A_avg3,B_avg3,C_avg3
2016-07-01,NaN,NaN,NaN
2016-07-02,NaN,NaN,NaN
2016-07-03,717.666667,3099.0,1345.0


In [15]:
# 3期間の移動平均を算出
x_avg3 = x.rolling(window=3, center=True).mean()  # 3 期間の中心で取る
x_avg3.columns = ['A_avg3', 'B_avg3', 'C_avg3']
x_avg3.head(3)

,A_avg3,B_avg3,C_avg3
2016-07-01,NaN,NaN,NaN
2016-07-02,717.666667,3099.000000,1345.000000
2016-07-03,852.666667,3811.333333,1625.333333


In [16]:
# 3 期間の最大値
x_max3 = x.rolling(window=3).max()
x_max3.columns = ['A_max3', 'B_max3', 'C_max3']
x_max3.head(3)

,A_max3,B_max3,C_max3
2016-07-01,NaN,NaN,NaN
2016-07-02,NaN,NaN,NaN
2016-07-03,823.0,3522.0,1711.0


`gropby()` メソッドや `resample()` メソッドと同じく、複数の統計量で集計する際は `agg()` メソッドを使用すると簡単です。

In [17]:
# 3 期間の平均値と最大値
x_avg3_max3 = x.rolling(window=3).agg(['mean', 'max'])
x_avg3_max3.head(3)

id                   A              B               C        
                  mean    max    mean     max    mean     max
2016-07-01         NaN    NaN     NaN     NaN     NaN     NaN
2016-07-02         NaN    NaN     NaN     NaN     NaN     NaN
2016-07-03  717.666667  823.0  3099.0  3522.0  1345.0  1711.0

## エキスパディング特徴量

**エクスパンディング特徴量**とは、**過去すべての期間の集計値**を特徴量としたものです。



In [18]:
# 元のデータを確認
x.head(3)

id,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711


In [19]:
# 過去全期間の平均値（当期を含む）
x_exp = x.expanding().mean()
x_exp.head(3)

id,A,B,C
2016-07-01,532.000000,3314.0,1136.0
2016-07-02,665.000000,2887.5,1162.0
2016-07-03,717.666667,3099.0,1345.0


In [20]:
# 過去全期間の平均値（前期まで）
x_exp = x.shift(1).expanding().mean()
x_exp.head(3)

id,A,B,C
2016-07-01,NaN,NaN,NaN
2016-07-02,532.0,3314.0,1136.0
2016-07-03,665.0,2887.5,1162.0


## 特徴量の結合

作成した特徴量を元のデータフレームと結合する方法をお伝えします。

In [21]:
# 元のデータフレーム
x.head(3)

id,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711


In [22]:
# ラグ特徴量
x_lag1.head(3)

,A_lag1,B_lag1,C_lag1
2016-07-01,NaN,NaN,NaN
2016-07-02,532.0,3314.0,1136.0
2016-07-03,798.0,2461.0,1188.0


In [23]:
# 特徴量の結合
x_concat = pd.concat([x, x_lag1], axis=1)
x_concat.head(3)

,A,B,C,A_lag1,B_lag1,C_lag1
2016-07-01,532,3314,1136,NaN,NaN,NaN
2016-07-02,798,2461,1188,532.0,3314.0,1136.0
2016-07-03,823,3522,1711,798.0,2461.0,1188.0


上記のように結合することができました。また `2016-07-01` 時点でのデータには欠損が発生しているため、欠損の削除または補完が必要な点も押さえておきましょう。
